In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Training a Shallow Text Classifier with TFIDF Preprocessing

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/carted/handling-variable-length-text-tf/blob/main/bigram-tfidf-shallow-mlp.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/carted/handling-variable-length-text-tf/blob/main/bigram-tfidf-shallow-mlp.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Setup

In [1]:
!gdown --id 1CvkRnGC8b_-n1NcbwcwxcIq7SusmDMb5 -O train_data.txt
!gdown --id 1h1evGF5NVi2p8RoWxl8xhpOod0ZN_-ky -O test_data_solution.txt 

Downloading...
From: https://drive.google.com/uc?id=1CvkRnGC8b_-n1NcbwcwxcIq7SusmDMb5
To: /content/train_data.txt
100% 35.4M/35.4M [00:00<00:00, 165MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1h1evGF5NVi2p8RoWxl8xhpOod0ZN_-ky
To: /content/test_data_solution.txt
100% 35.4M/35.4M [00:00<00:00, 165MB/s] 


In [1]:
from tensorflow import keras
import tensorflow as tf

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import random
import tqdm

SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

## Data loading

In [2]:
train_df = pd.read_csv(
    "train_data.txt",
    engine="python",
    sep=" ::: ",
    names=["id", "movie", "genre", "summary"],
)

test_df = pd.read_csv(
    "test_data_solution.txt",
    engine="python",
    sep=" ::: ",
    names=["id", "movie", "genre", "summary"],
)

## Data splitting

In [3]:
# Split the data using train_test_split from sklearn
train_shuffled = train_df.sample(frac=1)
train_df_new, val_df = train_test_split(train_shuffled, test_size=0.1)

print(f"Number of training samples: {len(train_df_new)}.")
print(f"Number of validation samples: {len(val_df)}.")
print(f"Number of test examples: {len(test_df)}.")

Number of training samples: 48792.
Number of validation samples: 5422.
Number of test examples: 54200.


## Data preprocessing

In [4]:
train_df_new["total_words"] = train_df_new["summary"].apply(lambda x: len(x.split()))
max_seqlen = train_df_new["total_words"].max()
max_seqlen

1829

In [5]:
text_vectorizer = keras.layers.TextVectorization(max_tokens=max_seqlen, ngrams=2, output_mode="tf_idf")
with tf.device("/CPU:0"):
    text_vectorizer.adapt(train_df_new["summary"])

label_encoder = keras.layers.StringLookup(vocabulary=train_df_new["genre"].unique())
label_encoder.get_vocabulary()

['[UNK]',
 'short',
 'sci-fi',
 'documentary',
 'drama',
 'thriller',
 'comedy',
 'adult',
 'romance',
 'adventure',
 'western',
 'family',
 'talk-show',
 'news',
 'horror',
 'history',
 'music',
 'sport',
 'war',
 'animation',
 'game-show',
 'action',
 'crime',
 'reality-tv',
 'mystery',
 'musical',
 'fantasy',
 'biography']

In [6]:
batch_size = 64
auto = tf.data.AUTOTUNE


def prepare_dataset(dataframe):
    dataset = tf.data.Dataset.from_tensor_slices(
        (dataframe["summary"], dataframe["genre"])
    )
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(
        lambda summaries, genres: (text_vectorizer(summaries), label_encoder(genres)),
        num_parallel_calls=auto,
    ).cache()
    return dataset.prefetch(auto)

In [7]:
training_dataset = prepare_dataset(train_df_new)
validation_dataset = prepare_dataset(val_df)
test_dataset = prepare_dataset(test_df)


for sequences, labels in training_dataset.take(1):
    print(sequences.shape, labels.shape)

(64, 1829) (64,)


## Model utilities

In [8]:
def make_model():
    shallow_mlp_model = keras.Sequential(
        [
            keras.layers.Dense(512, activation="relu"),
            keras.layers.Dense(256, activation="relu"),
            keras.layers.Dense(label_encoder.vocabulary_size(), activation="softmax"),
        ]
    )
    return shallow_mlp_model

## Training and evaluation

In [9]:
epochs = 60

shallow_mlp_model = make_model()
shallow_mlp_model.compile(
    loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)

history = shallow_mlp_model.fit(
    training_dataset, validation_data=validation_dataset, epochs=epochs
)

_, accuracy = shallow_mlp_model.evaluate(test_dataset)
print(f"Top-1 accuracy on the test set: {round(accuracy * 100, 2)}%.")

Epoch 1/60
763/763 [==============================] - 3s 3ms/step - loss: 2.1269 - accuracy: 0.4803 - val_loss: 1.6482 - val_accuracy: 0.5378
Epoch 2/60
763/763 [==============================] - 2s 2ms/step - loss: 1.4933 - accuracy: 0.5595 - val_loss: 1.5942 - val_accuracy: 0.5397
Epoch 3/60
763/763 [==============================] - 2s 2ms/step - loss: 1.2850 - accuracy: 0.6037 - val_loss: 1.6945 - val_accuracy: 0.5284
Epoch 4/60
763/763 [==============================] - 2s 2ms/step - loss: 1.1200 - accuracy: 0.6457 - val_loss: 1.7306 - val_accuracy: 0.5330
Epoch 5/60
763/763 [==============================] - 2s 2ms/step - loss: 0.9507 - accuracy: 0.6941 - val_loss: 1.8665 - val_accuracy: 0.5059
Epoch 6/60
763/763 [==============================] - 2s 2ms/step - loss: 0.8176 - accuracy: 0.7357 - val_loss: 2.1472 - val_accuracy: 0.4921
Epoch 7/60
763/763 [==============================] - 2s 2ms/step - loss: 0.7085 - accuracy: 0.7665 - val_loss: 2.5042 - val_accuracy: 0.5033
Epoch 